# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [69]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
#from census import Census

# Import API key
from api_keys import g_key

# # Access maps with unique API key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [70]:
# Output File (CSV)
city_data = pd.read_csv("../output_data/cities.csv")
city_data = city_data.dropna()
city_data.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,6165406,thompson,CA,1644099168,55.7435,-97.8558,-7.47,86,100,4.74
1,6089245,norman wells,CA,1644099169,65.2820,-126.8329,-3.51,90,97,6.85
2,4030556,rikitea,PF,1644099750,-23.1203,-134.9692,79.09,73,96,10.54
3,546105,nikolskoye,RU,1644098988,59.7035,30.7861,26.87,95,100,8.19
4,2136825,isangel,VU,1644099162,-19.5500,169.2667,82.51,89,75,9.22


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [71]:
# Plot Heatmap
#fig = gmaps.figure()
fig = gmaps.figure(center =[25,0],zoom_level=1.5)

location_df = city_data[["Lat","Lng"]]
heat_layer = gmaps.heatmap_layer(location_df, 
                                weights=city_data["Humidity"], 
                                dissipating = False,
                                max_intensity = 10,
                                point_radius = 1,
                                )

# Add the layer to the map
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [72]:
weather_criteria = city_data[
    (city_data['Max Temp'] < 80) & 
    (city_data['Max Temp'] >70) & 
    (city_data['Wind Speed'] < 10) & 
    (city_data['Cloudiness'] == 0)]

weather_criteria = weather_criteria.dropna()

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [73]:
hotel_df = weather_criteria
hotel_df['Hotel Name'] = " "
hotel_df['Hotel Address'] = " "
hotel_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Hotel Name,Hotel Address
23,2395568,aplahoue,BJ,1644099755,6.9333,1.6833,77.40,71,0,4.83,,
68,4021858,guerrero negro,MX,1644099171,27.9769,-114.0611,70.09,22,0,7.52,,
100,5848280,kapaa,US,1644099775,22.0752,-159.3190,79.48,67,0,1.99,,
227,3929631,san isidro,PE,1644099811,-12.1167,-77.0500,76.01,71,0,5.75,,
240,3932145,pisco,PE,1644099662,-13.7000,-76.2167,74.01,83,0,8.05,,


In [74]:
for index, row in hotel_df.iterrows():
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location": f"{row['Lat']}, {row['Lng']}",
        "type": "lodging",
        "radius": "5000",
        "key": g_key
        }
    #pprint(params)
    
    places_data = requests.get(base_url, params=params).json()

    try:
        hotel_df.loc[index, 'Hotel Name'] = places_data["results"][0]["name"]
        hotel_df.loc[index, 'Hotel Address'] = places_data["results"][0]["vicinity"]
    
    except Exception as error:
        print(error)
        
#pprint(places_data)

In [75]:
hotel_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Hotel Name,Hotel Address
23,2395568,aplahoue,BJ,1644099755,6.9333,1.6833,77.40,71,0,4.83,Hotel ARNOGOS,"RN2, Azove"
68,4021858,guerrero negro,MX,1644099171,27.9769,-114.0611,70.09,22,0,7.52,"Casa Laguna, Bed & Breakfast","Baja California, El Domino, Centro, Guerrero N..."
100,5848280,kapaa,US,1644099775,22.0752,-159.3190,79.48,67,0,1.99,Sheraton Kauai Coconut Beach Resort,"650 Aleka Loop, Kapaʻa"
227,3929631,san isidro,PE,1644099811,-12.1167,-77.0500,76.01,71,0,5.75,Soul Mate-Inn,"Toribio Pacheco 350, Miraflores"
240,3932145,pisco,PE,1644099662,-13.7000,-76.2167,74.01,83,0,8.05,Hotel San Isidro Oficial,"Calle, Pisco"
276,2425791,kibala,TD,1644099825,9.1104,18.3463,73.35,35,0,0.00,hotel safari,"49R9+F7X, Sarh"
359,286245,sur,OM,1644099847,22.5667,59.5289,71.01,54,0,6.80,Sur Plaza Hotel,Oman


In [76]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [77]:
fig = gmaps.figure(center =[25,0],zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations,
                                info_box_content = hotel_info)
fig.add_layer(marker_layer)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))